# Factor 

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from pathlib import Path
import numpy as np
import pandas as pd
import quantstats as qs
import statsmodels.api as sm
from linearmodels.asset_pricing import LinearFactorModel
from statsmodels.regression.rolling import RollingOLS
from linearmodels.panel import PanelOLS, FamaMacBeth
import wrds

%config InlineBackend.figure_format = "retina"

## Retrive Data

In [4]:
def cache(func):
    def wrapped_func(*args, **kwargs):
        table_name = func.__name__.split("_")[-1]
        parquet_path = f"./data/{table_name}.parquet"
        try:
            table = pd.read_parquet(parquet_path)
        except FileNotFoundError:
            table = func(*args, **kwargs)
            table.to_parquet(parquet_path, index=False)
        return table

    return wrapped_func

def query(db, sql_stmt, **kwargs):
    """
    Query WRDS database, with annoying loading library info suppressed.

    References:
    https://stackoverflow.com/questions/8391411/how-to-block-calls-to-print
    """
    data = db.raw_sql(sql_stmt, date_cols=["date"], params=kwargs)
    return data

def get_crsp(db, permnos):
    """
    Get price data from CRSP.

    Args:
        permnos: an iterative of permnos
    """
    sql_crsp = """
    SELECT
        date,
        permno,
        openprc AS open,
        askhi AS high,
        bidlo AS low,
        prc AS close,
        vol AS volume,
        ret,
        shrout
    FROM
        crsp.dsf
    WHERE
        permno IN %(permnos)s
        AND date >= '1994-01-01'
        AND date <= '2019-12-31'
    ORDER BY
        date, permno;
    """
    # Type check and transform permnos to tuple of str
    if isinstance(permnos, pd.DataFrame):
        permnos = permnos.permno
    if isinstance(permnos, pd.Series):
        permnos = permnos.astype("int").astype("str").pipe(tuple)
    elif any(not isinstance(permno, str) for permno in permnos):
        permnos = tuple(str(permno) for permno in permnos)
    elif not isinstance(permnos, tuple):
        permnos = tuple(permnos)
            
    # Query data
    crsp = query(db, sql_crsp, permnos=permnos)
    # Fill missing close prices of permno 80539
    crsp.loc[crsp.permno == 80539, "close"] = crsp.loc[crsp.permno == 80539, "close"].fillna(method="ffill")
    # Fill other missing values
    crsp = crsp.fillna({"open": crsp.close,
        "high": crsp.close,
        "low": crsp.close,
        "volume": 0,
        "ret": 0})
    # Calculate market capitalization
    crsp["cap"] = crsp.close * crsp.shrout
    # Shift market capitalization to avoid look ahead bias
    crsp["cap"] = crsp.groupby("permno").cap.shift(1)
    # Calculate market capiticalization weight
    crsp["w_cap"] = crsp.groupby("date").cap.apply(lambda x: x / x.sum())
    # Convert certain data types to int64
    crsp = crsp.astype({"permno":"int",
                        "shrout": "int",
                        "volume":"int"})
    return crsp

def get_optionmetrics(db, permnos):
    import concurrent.futures

    # Type check and transform permnos to tuple of str
    if isinstance(permnos, pd.DataFrame):
        permnos = permnos.permno
    if isinstance(permnos, pd.Series):
        permnos = permnos.astype("int").astype("str").pipe(tuple)
    elif any(not isinstance(permno, str) for permno in permnos):
        permnos = tuple(str(permno) for permno in permnos)
    elif not isinstance(permnos, tuple):
        permnos = tuple(permnos)

    sql_option_metrics = """
    SELECT
        date,
        permno,
        crsp.secid,
        iv,
        skew_1,
        skew_2
    FROM (
        SELECT
            *
        FROM
            wrdsapps.opcrsphist
        WHERE
            permno IN %(permnos)s) AS crsp
        JOIN (
        SELECT
            a.date,
            a.secid,
            c.iv,
            a.iv - b.iv AS skew_1,
            (g.iv - v.iv) / a.iv AS skew_2
        FROM (
            SELECT
                date,
                secid,
                impl_volatility AS iv
            FROM
                optionm.vsurfd%(year)s
            WHERE
                days = 30
                AND delta = 50) AS a,
            (
            SELECT
                date, secid, impl_volatility AS iv
            FROM
                optionm.vsurfd%(year)s
            WHERE
                days = 30
                AND delta = - 10) AS b,
            (
            SELECT
                date, secid, AVG(impl_volatility) AS iv
            FROM
                optionm.vsurfd%(year)s
            WHERE
                days = 30
                AND abs(delta) <= 50
            GROUP BY
                date,
                secid) AS c,
            (
            SELECT
                date,
                secid,
                impl_volatility AS iv
            FROM
                optionm.vsurfd%(year)s
            WHERE
                days = 30
                AND delta = - 25) AS g,
            (
            SELECT
                date, secid, impl_volatility AS iv
            FROM
                optionm.vsurfd%(year)s
            WHERE
                days = 30
                AND delta = 25) AS v
            WHERE
                a.date = b.date
                AND a.secid = b.secid
                AND a.date = c.date
                AND a.secid = c.secid
                AND a.date = g.date
                AND a.secid = g.secid
                AND a.date = v.date
                AND a.secid = v.secid) AS om ON crsp.secid = om.secid;
    """
    
    optionmetrics = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_year = [
            executor.submit(
                lambda year: query(db, sql_option_metrics, year=year, permnos=permnos),
                year,
            )
            for year in range(2000, 2020)
        ]
        for future in concurrent.futures.as_completed(future_to_year):
            optionmetrics.append(future.result())
    # Convert certain data types to int64
    optionmetrics = pd.concat(optionmetrics).sort_values(["date", "permno"]).astype({"permno": "int", "secid": "int"})
    return optionmetrics

def get_mfis(permnos):
    # Type check and transform permnos to tuple of int
    if isinstance(permnos, pd.DataFrame):
        permnos = permnos.permno
    if isinstance(permnos, pd.Series):
        permnos = permnos.astype("int").pipe(tuple)
    elif any(not isinstance(permno, int) for permno in permnos):
        permnos = tuple(str(permno) for permno in permnos)
    elif not isinstance(permnos, tuple):
        permnos = tuple(permnos)
    # Download MFIS
    mfis = pd.read_csv("https://files.osf.io/v1/resources/btvdh/providers/dropbox/1996-2019/MFIS_1996_2019.csv",
        names=["date", "permno", "mfis_30", "mfis_91", "mfis_182", "mfis_273", "mfis_365"],
        header=0,
        parse_dates=["date"],
        dtype={"permno":"int"})
    mfis = mfis[mfis.permno.isin(permnos)]
    return mfis

def get_glb(permnos):
    # Type check and transform permnos to tuple of int
    if isinstance(permnos, pd.DataFrame):
        permnos = permnos.permno
    if isinstance(permnos, pd.Series):
        permnos = permnos.astype("int").pipe(tuple)
    elif any(not isinstance(permno, int) for permno in permnos):
        permnos = tuple(str(permno) for permno in permnos)
    elif not isinstance(permnos, tuple):
        permnos = tuple(permnos)
    # Download GLB
    glb = pd.read_csv("https://files.osf.io/v1/resources/7xcqw/providers/dropbox/1996-2019/glb30_daily.csv",
        names=["permno", "date", "glb2_30", "glb3_30"],
        header=0,
        parse_dates=["date"],
        dtype={"permno":"int"})
    glb = glb[glb.permno.isin(permnos)]
    return glb 

def get_famafrench():
    """
    Get 3-factor, momentum, industry portfolio return from Ken French data library.
    """
    import pandas_datareader as web

    # Transfrom from percentage to nominal value
    factor = web.DataReader("F-F_Research_Data_Factors_daily", "famafrench", start="1994-01-01", end="2019-12-31")[0]/100
    momentum = web.DataReader("F-F_Momentum_Factor_daily", "famafrench", start="1994-01-01", end="2019-12-31")[0]/100
    industry = web.DataReader("17_Industry_Portfolios_daily", "famafrench", start="1994-01-01", end="2019-12-31")[0]/100
    # Substract the risk free rate
    momentum = momentum.sub(factor.RF, axis=0)
    industry = industry.sub(factor.RF, axis=0)
    # Merge into one dataframe
    fama_french = pd.concat([factor, momentum, industry], axis=1)
    fama_french = fama_french.rename(columns=lambda x: x.lower().strip().replace("-", "")).rename_axis(index=str.lower)
    return fama_french

In [ ]:
def get_compustat(wrds_username):
    """
    SELECT
        CASE WHEN fdate < datadate + '6 months'::INTERVAL THEN
            fdate + '3 days'::INTERVAL
        ELSE
            datadate + '3 months'::INTERVAL END::DATE AS date,
        permno,
        sich AS sic,
        naicsh,
        *    
    FROM (
        SELECT
            lpermno AS permno,
            gvkey
        FROM
            crsp.ccmxpf_lnkhist
        WHERE
            lpermno IN ('10874', '11308', '11404', '11786', '13856', '16432', '16600', '17144', '17750', '18163', '18411', '18729', '20482', '21207', '21776', '22111', '22752', '24010', '24109', '24205', '25320', '26825', '27959', '42200', '43449', '45911', '46578', '47466', '47626', '48486', '51369', '52476', '53065', '53613', '56274', '61241', '64936', '69032', '71298', '75175', '75341', '76644', '77649', '80539', '82598', '83601', '84519', '85663', '86102', '88664')
            AND linkprim IN ('P', 'C')
            AND linktype IN ('LU', 'LC')
            AND linkenddt IS NULL) AS link
        JOIN (
            SELECT
                *
            FROM
                comp.funda
            WHERE
                fyear = 2018
                AND indfmt = 'INDL'
                AND datafmt = 'STD'
                AND popsrc = 'D'
                AND consol = 'C') AS fund ON link.gvkey = fund.gvkey LIMIT 100;
    """

In [133]:
db = wrds.Connection(wrds_username="iewaij")

Loading library list...
Done


In [154]:
path = Path("./data")
permno_path = path/"permno_selection.csv"
permnos = pd.read_csv(permno_path, dtype={"permno":"str"}).squeeze()
crsp = get_crsp(db, permnos=permnos)
optionmetrics = get_optionmetrics(db, permnos=permnos)
mfis = get_mfis(permnos=permnos)
glb = get_glb(permnos=permnos)
famafrench = get_famafrench()

In [5]:
db.close()

In [7]:
# mfis["date"] = pd.to_datetime(mfis["date"])
# glb["date"] =pd.to_datetime(glb["date"])
# mfis.to_parquet("./data/mfis.parquet", index=False)
# glb.to_parquet("./data/glb.parquet", index=False)
famafrench.to_parquet("./data/famafrench.parquet", index=True)


## Linear Factor Model

In [215]:
def test_factor(crsp, famafrench):
    portfolio = crsp.pivot(index="date", columns="permno", values="ret")
    portfolio = portfolio.sub(famafrench.rf, axis=0)["2000-01-01":]
    factor = famafrench.drop(columns="rf")["2000-01-01":]
    mod = LinearFactorModel(portfolios=portfolio, factors=factor)
    return  mod.fit().summary

In [216]:
test_factor(crsp, famafrench)

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                      LinearFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 50   R-squared:                      0.4514
No. Factors:                         21   J-statistic:                    9.2138
No. Observations:                  5031   P-value                         0.9998
Date:                  Tue, Mar 29 2022   Distribution:                 chi2(29)
Time:                          16:18:03                                         
Cov. Estimator:                  robust                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
mktrf          0.0004     0.0002     1.6480     0.0994  -7.086e-05      0.0008
smb            0.0004     0.0003     1.3545     0.1756     -0.0002      0.0010
hml           -0.0004     0.0003    -1.2576     0.2085     -0.0011      0.0002
mom            0.0003     0.0010     0.3606     0.7184     -0.0015      0.0022
food           0.0004     0.0002     2.2106     0.0271   3.999e-05      0.0007
mines          0.0005     0.0005     1.1764     0.2394     -0.0004      0.0014
oil            0.0010     0.0010     0.9783     0.3279     -0.0010      0.0030
clths          0.0015     0.0014     1.0534     0.2921     -0.0013      0.0042
durbl          0.0006     0.0007     0.9666     0.3337     -0.0007      0.0019
chems       1.551e-05     0.0014     0.0111     0.9911     -0.0027      0.0027
cnsum          0.0003     0.0002     2.0019     0.0453   7.283e-06      0.0007
cnstr          0.0008     0.0010     0.8160     0.4145     -0.0012      0.0028
steel          0.0003     0.0004     0.7824     0.4340     -0.0005      0.0012
fabpr          0.0010     0.0008     1.2134     0.2250     -0.0006      0.0026
machn          0.0006     0.0003     2.0628     0.0391   2.948e-05      0.0012
cars           0.0004     0.0009     0.3933     0.6941     -0.0015      0.0022
trans          0.0006     0.0006     0.8881     0.3745     -0.0007      0.0018
utils          0.0005     0.0002     2.6813     0.0073      0.0001      0.0009
rtail          0.0007     0.0005     1.4830     0.1381     -0.0002      0.0016
finan          0.0006     0.0003     1.9794     0.0478   5.895e-06      0.0012
other          0.0001     0.0003     0.3477     0.7280     -0.0006      0.0008
==============================================================================

Covariance estimator:
HeteroskedasticCovariance
See full_summary for complete results
"""

## Estimate Factor Exposure and Characteristics

In [244]:
def combine_factor(freq, **kwargs):
    factor = pd.DataFrame(
        {
            name: factor.resample(freq).last().stack()
            for name, factor in kwargs.items()
        }
    )
    return factor

def calc_factor(crsp, famafrench, mfis, glb, freq):
    from statsmodels.regression.rolling import RollingOLS
    
    # Estimate factor exposure
    portfolio = crsp.loc[crsp.date >= "1994-01-01", :].pivot(index="date", columns="permno", values="ret")
    portfolio = portfolio.sub(famafrench.loc[portfolio.index, "rf"], axis=0)
    factor = famafrench.loc[portfolio.index, ["mktrf", "smb", "hml", "mom"]].assign(alpha=1)
    betas = []
    for permno in portfolio:
        ret = portfolio[permno]
        res = RollingOLS(endog=ret, exog=factor, window=756).fit(params_only=True)
        params = res.params
        params["resid"] = ret - factor.mul(params).sum(1)
        params["permno"] = permno 
        betas.append(params)
    beta = pd.concat(betas).reset_index().dropna()
    
    # Pivot data for calculation
    ret = crsp.pivot(index="date", columns="permno", values="ret").sub(famafrench.rf, axis=0)
    logret = np.log(ret+1)
    shrout = crsp.pivot(index="date", columns="permno", values="shrout")
    vol = crsp.pivot(index="date", columns="permno", values="volume")
    close = crsp.pivot(index="date", columns="permno", values="close") 
    dv = vol.mul(close)
    mktrf = beta.pivot(index="date", columns="permno", values="mktrf")
    resid = beta.pivot(index="date", columns="permno", values="resid")

    # Estimate price trend
    # 1-month cumulative return
    mom_1m = logret.rolling(21).sum()
    # 11-month cumulative returns ending 1-month before
    mom_12m = logret.shift(21).rolling(11*21).sum()
    # Cumulative return from months t-6 to t-1 minus months t-12 to t-7.
    mom_6m = logret.shift(21).rolling(5*21).sum()
    mom_12m_6m = logret.shift(6*21).rolling(5*21).sum()
    chmom = mom_6m - mom_12m_6m
    # Max daily returns from calendar month t-1
    maxret = logret.rolling(21).max()
    # Cumulative returns months t-36 to t-13
    mom_36m = logret.shift(12*21).rolling(24*21).sum()

    # Estimate liquidity
    # Average monthly trading volume for most recent three months divided by number of shares
    turn = vol.shift(21).mean().div(shrout)
    # Monthly std dev of daily share turnover
    turn_std = vol.div(shrout).rolling(21).std()
    # Natural log of market cap
    logcap = np.log(close) + np.log(shrout)
    # Natural log of trading volume times price per share from month t-2
    dolvol = np.log(dv.shift(21).rolling(21).mean())
    # Average of daily (absolute return / dollar volume)
    ill = ret.abs().div(dv)

    # Estimate risk
    # Standard dev of daily returns from month t-1
    retvol = ret.rolling(21).std()
    # Market beta squared
    mktrf_sq = mktrf.pow(2)
    # Idiosyncratic return volatility
    idovol = resid.rolling(756).std()

    # Combine factors to the required frequency
    factor = combine_factor(freq,
        mom_1m=mom_1m,
        mom_12m=mom_12m,
        chmom=chmom,
        maxret=maxret,
        mom_36m=mom_36m,
        turn=turn,
        turn_std=turn_std,
        logcap=logcap,
        dolvol=dolvol,
        ill=ill,
        retvol=retvol,
        mktrf_sq=mktrf_sq,
        idovol=idovol
    )
    if freq == "D":
        beta_ = beta.set_index(["date", "permno"])
        mfis_ = mfis.set_index(["date", "permno"])
        glb_ = glb.set_index(["date", "permno"])
    else:
        beta_ = beta.groupby([pd.Grouper(key="date", freq=freq), "permno"]).mean()
        mfis_ = mfis.groupby([pd.Grouper(key="date", freq=freq), "permno"]).mean()
        glb_ = glb.groupby([pd.Grouper(key="date", freq=freq), "permno"]).mean()
    factor = factor.join(beta_).join(mfis_).join(glb_)
    
    # Fill missing value with cross sectional mean
    factor = factor.groupby("date").transform(lambda x: x.fillna(x.mean()))
    return factor

def calc_return(crsp, famafrench, freq):
    if freq == "D":
        return crsp.set_index(["date", "permno"]).ret
    else:
        ret = crsp.pivot(index="date", columns="permno", values="ret").sub(famafrench.rf, axis=0)
        logret = np.log(ret+1)
        return np.exp(logret.resample(freq).sum().stack()) - 1    

In [246]:
factor = calc_factor(crsp, famafrench, "W-FRI")
ret = calc_return(crsp, famafrench, "W-FRI")

/opt/homebrew/Caskroom/mambaforge/base/envs/quant/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


## Panel Regression

In [248]:
X = factor["2000-01-01":"2012-12-31"] 
y = ret.loc["2000-01-01":"2012-12-31"]
dependent = y.swaplevel()
exog = sm.add_constant(X.swaplevel())
mod = FamaMacBeth(dependent, exog)
res = mod.fit(cov_type='unadjusted')
res

Dep. Variable:,0,R-squared:,0.6334
Estimator:,FamaMacBeth,R-squared (Between):,0.6619
No. Observations:,33900,R-squared (Within):,0.6333
Date:,"Tue, Mar 29 2022",R-squared (Overall):,0.6334
Time:,16:33:30,Log-likelihood,6.523e+04
Cov. Estimator:,Fama-MacBeth Standard Cov,,
,,F-statistic:,2250.5
Entities:,50,P-value,0.0000
Avg Obs:,678.00,Distribution:,"F(26,33873)"
Min Obs:,678.00,,
Max Obs:,678.00,F-statistic (robust):,3463.8


In [249]:
X = factor["2000-01-01":"2012-12-31"] 
y = ret.shift(-1).loc["2000-01-01":"2012-12-31"]
dependent = y.swaplevel()
exog = sm.add_constant(X.swaplevel())
mod = FamaMacBeth(dependent, exog)
res = mod.fit(cov_type='unadjusted')
res

Dep. Variable:,0,R-squared:,-0.0626
Estimator:,FamaMacBeth,R-squared (Between):,-13.981
No. Observations:,33900,R-squared (Within):,-0.0582
Date:,"Tue, Mar 29 2022",R-squared (Overall):,-0.0626
Time:,16:33:33,Log-likelihood,4.719e+04
Cov. Estimator:,Fama-MacBeth Standard Cov,,
,,F-statistic:,-76.779
Entities:,50,P-value,1.0000
Avg Obs:,678.00,Distribution:,"F(26,33873)"
Min Obs:,678.00,,
Max Obs:,678.00,F-statistic (robust):,1.3622


## OLS

In [250]:
# In-sample hist return OLS
X = factor["2000-01-01":"2012-12-31"] 
y = ret.loc["2000-01-01":"2012-12-31"]
sm.OLS(endog=y, exog=X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.674
Model:                            OLS   Adj. R-squared (uncentered):              0.674
Method:                 Least Squares   F-statistic:                              2698.
Date:                Tue, 29 Mar 2022   Prob (F-statistic):                        0.00
Time:                        16:33:54   Log-Likelihood:                          67225.
No. Observations:               33900   AIC:                                 -1.344e+05
Df Residuals:                   33874   BIC:                                 -1.342e+05
Df Model:                          26                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
mom_1m         0.1174      0.002     58.232      0.000       0.113       0.121
mom_12m        0.0032      0.001      4.765      0.000       0.002       0.005
chmom          0.0021      0.000      4.248      0.000       0.001       0.003
maxret        -0.1888      0.014    -13.881      0.000      -0.215      -0.162
mom_36m        0.0021      0.001      3.768      0.000       0.001       0.003
turn       -3.149e-05   1.82e-05     -1.727      0.084   -6.72e-05    4.26e-06
turn_std    -4.22e-05   3.79e-05     -1.114      0.265      -0.000     3.2e-05
logcap        -0.0003      0.000     -0.764      0.445      -0.001       0.000
dolvol        -0.0002      0.000     -0.638      0.524      -0.001       0.000
ill        -1.532e+05   9.08e+04     -1.687      0.092   -3.31e+05    2.48e+04
retvol         0.6547      0.038     17.407      0.000       0.581       0.728
mktrf_sq      -0.0008      0.001     -1.007      0.314      -0.002       0.001
idovol         0.1053      0.037      2.819      0.005       0.032       0.179
mktrf          0.0009      0.002      0.585      0.558      -0.002       0.004
smb           -0.0011      0.001     -2.128      0.033      -0.002   -8.96e-05
hml           -0.0014      0.000     -3.973      0.000      -0.002      -0.001
mom           -0.0007      0.001     -1.023      0.306      -0.002       0.001
alpha          0.1694      0.444      0.382      0.703      -0.700       1.039
resid          4.4038      0.021    214.576      0.000       4.364       4.444
mfis_30       -0.0005      0.001     -0.989      0.323      -0.001       0.000
mfis_91       -0.0007      0.001     -0.705      0.481      -0.003       0.001
mfis_182     7.28e-05      0.001      0.050      0.960      -0.003       0.003
mfis_273      -0.0015      0.002     -0.635      0.526      -0.006       0.003
mfis_365      -0.0030      0.002     -1.467      0.143      -0.007       0.001
glb2_30        0.4054      0.147      2.754      0.006       0.117       0.694
glb3_30       -0.4059      0.146     -2.775      0.006      -0.693      -0.119
==============================================================================
Omnibus:                     9790.147   Durbin-Watson:                   0.928
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           709578.214
Skew:                           0.478   Prob(JB):                         0.00
Kurtosis:                      25.393   Cond. No.                     1.40e+10
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenva

In [251]:
# In-sample forward return OLS
X = factor["2000-01-01":"2012-12-31"] 
y = ret.shift(-1).loc["2000-01-01":"2012-12-31"]
sm.OLS(endog=y, exog=X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.017
Model:                            OLS   Adj. R-squared (uncentered):              0.017
Method:                 Least Squares   F-statistic:                              22.96
Date:                Tue, 29 Mar 2022   Prob (F-statistic):                   2.91e-108
Time:                        16:33:59   Log-Likelihood:                          48502.
No. Observations:               33900   AIC:                                 -9.695e+04
Df Residuals:                   33874   BIC:                                 -9.673e+04
Df Model:                          26                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
mom_1m         0.0697      0.004     19.885      0.000       0.063       0.077
mom_12m        0.0010      0.001      0.813      0.416      -0.001       0.003
chmom          0.0014      0.001      1.675      0.094      -0.000       0.003
maxret        -0.1403      0.024     -5.939      0.000      -0.187      -0.094
mom_36m        0.0018      0.001      1.887      0.059   -7.05e-05       0.004
turn        7.039e-06   3.17e-05      0.222      0.824   -5.51e-05    6.91e-05
turn_std   -5.269e-06   6.58e-05     -0.080      0.936      -0.000       0.000
logcap        -0.0003      0.001     -0.417      0.677      -0.002       0.001
dolvol        -0.0002      0.001     -0.338      0.735      -0.001       0.001
ill        -1.615e+05   1.58e+05     -1.023      0.306   -4.71e+05    1.48e+05
retvol         0.4748      0.065      7.267      0.000       0.347       0.603
mktrf_sq      -0.0008      0.001     -0.599      0.549      -0.003       0.002
idovol         0.1828      0.065      2.817      0.005       0.056       0.310
mktrf          0.0016      0.003      0.570      0.569      -0.004       0.007
smb           -0.0024      0.001     -2.549      0.011      -0.004      -0.001
hml           -0.0010      0.001     -1.567      0.117      -0.002       0.000
mom        -6.286e-05      0.001     -0.056      0.956      -0.002       0.002
alpha         -2.3070      0.771     -2.993      0.003      -3.818      -0.796
resid         -0.0337      0.036     -0.944      0.345      -0.104       0.036
mfis_30       -0.0010      0.001     -1.158      0.247      -0.003       0.001
mfis_91       -0.0008      0.002     -0.486      0.627      -0.004       0.002
mfis_182       0.0004      0.003      0.167      0.868      -0.005       0.005
mfis_273       0.0024      0.004      0.562      0.574      -0.006       0.011
mfis_365      -0.0062      0.004     -1.751      0.080      -0.013       0.001
glb2_30       -0.6217      0.256     -2.432      0.015      -1.123      -0.121
glb3_30        0.6137      0.254      2.415      0.016       0.116       1.112
==============================================================================
Omnibus:                    10672.737   Durbin-Watson:                   1.635
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           352748.666
Skew:                           0.873   Prob(JB):                         0.00
Kurtosis:                      18.706   Cond. No.                     1.40e+10
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenva